In [1]:
# import numpy 
import numpy as np 

# import pandas
import pandas as pd 

# import regex
import re

# import nltk
import nltk 

In [2]:
# load data
data_source_url = "https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv"
airline_tweets = pd.read_csv(data_source_url)

* print top 5 rows

In [3]:
airline_tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


* use 'text' column to create array with name 'features'
* use 'airline_sentiment' column to create array with name 'labels'

In [5]:
features = np.array(airline_tweets.text)
features[0:25]

array(['@VirginAmerica What @dhepburn said.',
       "@VirginAmerica plus you've added commercials to the experience... tacky.",
       "@VirginAmerica I didn't today... Must mean I need to take another trip!",
       '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse',
       "@VirginAmerica and it's a really big bad thing about it",
       "@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA",
       '@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)',
       '@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP',
       "@virginamerica Well, I didn't…but NOW I DO! :-D",
       "@VirginAmerica it was amazing, and arrived an hour early. You're too good to me.",
       '@VirginAmerica did you know that suicide is the second leading cause of

* clean the text in features array
* Remove all the special characters
* remove all single characters
* Remove single characters from the start
* Substituting multiple spaces with single space
* Removing prefixed 'b'
* Converting to Lowercase

In [16]:
test = features[1]
test

"@VirginAmerica plus you've added commercials to the experience... tacky."

In [17]:
re.sub('\W+', ' ', test)

' VirginAmerica plus you ve added commercials to the experience tacky '

In [18]:
test

"@VirginAmerica plus you've added commercials to the experience... tacky."

In [27]:
processed_features = []

import string

def remove_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)

def lower_case(text):
    return text.lower()

for sentence in features:
    # Remove all the special characters
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)

    # remove all single characters
    sentence = ' '.join( [w for w in sentence.split() if len(w)>1] )

    # Remove single characters from the start
    

    # Substituting multiple spaces with single space
    sentence = re.sub('\s+',' ',sentence)

    # Removing prefixed 'b'
    

    # Converting to Lowercase
    sentence = lower_case(sentence)
    processed_features.append(sentence)

* import stopwords from nltk

In [29]:

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

* import TfidfVectorizer from sklearn

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))

* create vectorizer inastance of TfidfVectorizer with following settings:
* max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english')

* transform features with vectorizer

In [31]:
X = vectorizer.fit_transform(processed_features)

* import train_test_split from sklearn and split the data

In [37]:
#Creating the dependent variable class
factor = pd.factorize(airline_tweets['airline_sentiment'])
y = factor[0]
definitions = factor[1]
print(y[:5])
print(definitions)

[0 1 0 2 2]
Index(['neutral', 'positive', 'negative'], dtype='object')


In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

* import classifier of your choice from sklearn (e.g. Random Forest, LogReg, Naive Bayes)

In [39]:
from sklearn.ensemble import RandomForestClassifier

* fit your classifier to data

In [49]:
rf = RandomForestClassifier(n_estimators = 250, criterion = 'entropy', random_state = 0)
rf.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=250, random_state=0)

* predict X_test

In [50]:
y_pred = rf.predict(X_test)

* import confusion matrix and accuracy_score

In [43]:
from sklearn.metrics import roc_auc_score

* print confusion matrix

In [51]:
from sklearn import metrics
cf_matrix = metrics.confusion_matrix(y_test, y_pred)
print(cf_matrix)

[[ 314   52  386]
 [  69  339  195]
 [ 120   45 2140]]


* print accaccuracy_score

In [52]:
print('Accuracy score', metrics.accuracy_score(y_test,y_pred))

Accuracy score 0.7631147540983606
